# Universe and Data

### Setup

In [ ]:
import os
import time
from datetime import datetime, timedelta

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data sources (Phase 1: keep it simple)
import yfinance as yf

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

REPO_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))  # notebooks/ -> repo root
DATA_RAW = os.path.join(REPO_ROOT, "data", "raw")
DATA_PROCESSED = os.path.join(REPO_ROOT, "data", "processed")
DATA_CACHE = os.path.join(REPO_ROOT, "data", "cache")

for p in [DATA_RAW, DATA_PROCESSED, DATA_CACHE]:
    os.makedirs(p, exist_ok=True)

print("Repo root:", REPO_ROOT)
print("Raw:", DATA_RAW)
print("Processed:", DATA_PROCESSED)


### Universe definition

In [ ]:
# Start with a strong but manageable universe for Phase 1.
# Tip: keep it 20–50 tickers so notebooks run fast.
UNIVERSE = [
    "AAPL","MSFT","NVDA","AMZN","GOOGL","META","TSLA",
    "JPM","BAC","GS",
    "UNH","JNJ","PFE",
    "XOM","CVX",
    "KO","PEP",
    "COST","WMT",
    "SPY","QQQ"
]

# Time range
END = datetime.today().date()
START = END - timedelta(days=365*8)  # 8 years of daily data

print("Tickers:", len(UNIVERSE))
print("Start:", START, "End:", END)


### Download OHLCV (adjusted) + corporate actions

In [ ]:
def fetch_ohlcv(tickers, start, end, interval="1d"):
    """
    Returns a multi-index columns DataFrame from yfinance:
    columns like ('Close','AAPL'), etc.
    """
    df = yf.download(
        tickers=tickers,
        start=str(start),
        end=str(end),
        interval=interval,
        auto_adjust=False,  # we'll keep both raw + adjusted logic explicit
        group_by="column",
        threads=True,
        progress=False
    )
    return df

ohlcv_raw = fetch_ohlcv(UNIVERSE, START, END)
ohlcv_raw.head()


### Normalize OHLCV into a tidy table

In [ ]:
def tidy_ohlcv(ohlcv):
    """
    Convert yfinance multi-index OHLCV into tidy long-form dataframe.
    """
    # yfinance returns columns like: Open, High, Low, Close, Adj Close, Volume
    # with tickers on second level if multiple tickers.
    if isinstance(ohlcv.columns, pd.MultiIndex):
        tidy = (
            ohlcv
            .stack(level=1, future_stack=True)
            .reset_index()
            .rename(columns={"level_1":"ticker"})
        )
    else:
        # single ticker case
        tidy = ohlcv.reset_index()
        tidy["ticker"] = "SINGLE"
    
    
    tidy = tidy.rename(columns={
        "Date":"date",
        "Ticker":"ticker",
        "Open":"open",
        "High":"high",
        "Low":"low",
        "Close":"close",
        "Adj Close":"adj_close",
        "Volume":"volume"
    })
    tidy["date"] = pd.to_datetime(tidy["date"])
    tidy = tidy.sort_values(["ticker","date"]).reset_index(drop=True)
    return tidy

ohlcv = tidy_ohlcv(ohlcv_raw)
ohlcv.head()


### Basic data quality checks

In [ ]:
def quality_report(df):
    # missing values by column
    missing = df.isna().mean().sort_values(ascending=False)
    
    # duplicates
    dup = df.duplicated(subset=["date","ticker"]).sum()
    
    # per ticker coverage
    coverage = df.groupby("ticker")["date"].agg(["min","max","count"])
    return missing, dup, coverage

missing, dup, coverage = quality_report(ohlcv)
print("Duplicate rows (date,ticker):", dup)
display(missing.head(10))
display(coverage.sort_values("count"))


### Fundamentals snapshot

Fetching a small set of fundamentals per ticker:

- market cap
- trailing PE
- forward PE (if available)
- price-to-book
- profit margins
- revenue growth (if available)
- debt/equity (if available)
- dividend yield (if available)

In [ ]:
def fetch_fundamentals_snapshot(tickers, sleep_s=0.2):
    rows = []
    for t in tickers:
        try:
            info = yf.Ticker(t).info
            rows.append({
                "ticker": t,
                "asof": pd.Timestamp.utcnow(),
                "marketCap": info.get("marketCap"),
                "trailingPE": info.get("trailingPE"),
                "forwardPE": info.get("forwardPE"),
                "priceToBook": info.get("priceToBook"),
                "profitMargins": info.get("profitMargins"),
                "revenueGrowth": info.get("revenueGrowth"),
                "debtToEquity": info.get("debtToEquity"),
                "dividendYield": info.get("dividendYield"),
                "sector": info.get("sector"),
                "industry": info.get("industry"),
            })
            time.sleep(sleep_s)
        except Exception as e:
            rows.append({"ticker": t, "asof": pd.Timestamp.utcnow(), "error": str(e)})
    return pd.DataFrame(rows)

fundamentals = fetch_fundamentals_snapshot(UNIVERSE)
fundamentals.head()


### Save artifacts to file

In [ ]:
ohlcv_path = os.path.join(DATA_RAW, "ohlcv.parquet")
fund_path = os.path.join(DATA_RAW, "fundamentals_snapshot.parquet")

ohlcv.to_parquet(ohlcv_path, index=False)
fundamentals.to_parquet(fund_path, index=False)


print("Saved:", ohlcv_path)
print("Saved:", fund_path)


### Quick sanity plots

In [ ]:
# sanity plot: close price for SPY and QQQ
plot_df = ohlcv[ohlcv["ticker"].isin(["SPY","QQQ"])].pivot(index="date", columns="ticker", values="adj_close")

plot_df.plot(title="Adjusted Close (SPY vs QQQ)", figsize=(10,4))
plt.show()


### “Processed” dataset stub

In [ ]:
processed = ohlcv.merge(fundamentals.drop(columns=["asof"]), on="ticker", how="left")

processed_path = os.path.join(DATA_PROCESSED, "market_merged.parquet")
processed.to_parquet(processed_path, index=False)

processed.head(), processed_path
